In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold

from preprocessing import preproc
from tools import log_loss_lgbm

from datetime import datetime
import time

path_to_data = "/home/raph/Downloads/"

print("loading data")
training = pd.read_csv(path_to_data+"training.csv")
testing = pd.read_csv(path_to_data+"testing.csv")

loading data


In [9]:
testing.describe()

,actual_amount_paid,bd,city,gender,is_auto_renew,is_cancel,is_churn,payment_method_id,payment_plan_days,plan_list_price,registered_via,total_number_of_transactions,usual_price_per_day,price_per_day,price_per_day_diff
count,1.072753e+06,1.076941e+06,1.076941e+06,1.076941e+06,1.072753e+06,1.072753e+06,1076941.0,1.072753e+06,1.072753e+06,1.072753e+06,1.076941e+06,1.076941e+06,1.076941e+06,1.076941e+06,1.076941e+06
mean,1.637913e+02,1.132238e+01,5.173659e+00,-4.038392e-01,8.579813e-01,1.652104e-02,0.0,3.864096e+01,3.837042e+01,1.642593e+02,5.972834e+00,1.254605e+01,2.579224e+02,9.460549e+00,-2.484618e+02
std,2.074166e+02,1.606259e+01,6.456481e+00,7.900979e-01,3.490695e-01,1.274681e-01,0.0,4.036039e+00,4.838516e+01,2.072031e+02,3.098771e+00,7.691915e+00,1.042853e+03,4.739707e+02,1.055519e+03
min,0.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.0,2.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.770247e+05
25%,9.900000e+01,-1.000000e+00,1.000000e+00,-1.000000e+00,1.000000e+00,0.000000e+00,0.0,3.800000e+01,3.000000e+01,9.900000e+01,4.000000e+00,6.000000e+00,1.997780e+00,3.298900e+00,3.800231e-01
50%,1.490000e+02,-1.000000e+00,1.000000e+00,-1.000000e+00,1.000000e+00,0.000000e+00,0.0,4.100000e+01,3.000000e+01,1.490000e+02,7.000000e+00,1.300000e+01,2.767102e+00,4.965012e+00,1.302228e+00
75%,1.490000e+02,2.600000e+01,1.100000e+01,0.000000e+00,1.000000e+00,0.000000e+00,0.0,4.100000e+01,3.000000e+01,1.490000e+02,7.000000e+00,1.900000e+01,3.960396e+00,4.965012e+00,2.038531e+00
max,2.000000e+03,9.900000e+01,2.200000e+01,1.000000e+00,1.000000e+00,1.000000e+00,0.0,4.100000e+01,4.500000e+02,2.000000e+03,1.300000e+01,2.380000e+02,1.770297e+05,1.788000e+05,1.527562e+05


In [4]:
testing.describe()

,actual_amount_paid,bd,city,gender,is_auto_renew,is_cancel,is_churn,payment_method_id,payment_plan_days,plan_list_price,registered_via,total_number_of_transactions,usual_price_per_day,price_per_day,price_per_day_diff
count,1.072753e+06,1.076941e+06,1.076941e+06,1.076941e+06,1.072753e+06,1.072753e+06,1076941.0,1.072753e+06,1.072753e+06,1.072753e+06,1.076941e+06,1.076941e+06,1.076941e+06,1.076941e+06,1.076941e+06
mean,1.637913e+02,1.132238e+01,5.173659e+00,-4.038392e-01,8.579813e-01,1.652104e-02,0.0,3.864096e+01,3.837042e+01,1.642593e+02,5.972834e+00,1.254605e+01,2.579224e+02,9.460549e+00,-2.484618e+02
std,2.074166e+02,1.606259e+01,6.456481e+00,7.900979e-01,3.490695e-01,1.274681e-01,0.0,4.036039e+00,4.838516e+01,2.072031e+02,3.098771e+00,7.691915e+00,1.042853e+03,4.739707e+02,1.055519e+03
min,0.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.0,2.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.770247e+05
25%,9.900000e+01,-1.000000e+00,1.000000e+00,-1.000000e+00,1.000000e+00,0.000000e+00,0.0,3.800000e+01,3.000000e+01,9.900000e+01,4.000000e+00,6.000000e+00,1.997780e+00,3.298900e+00,3.800231e-01
50%,1.490000e+02,-1.000000e+00,1.000000e+00,-1.000000e+00,1.000000e+00,0.000000e+00,0.0,4.100000e+01,3.000000e+01,1.490000e+02,7.000000e+00,1.300000e+01,2.767102e+00,4.965012e+00,1.302228e+00
75%,1.490000e+02,2.600000e+01,1.100000e+01,0.000000e+00,1.000000e+00,0.000000e+00,0.0,4.100000e+01,3.000000e+01,1.490000e+02,7.000000e+00,1.900000e+01,3.960396e+00,4.965012e+00,2.038531e+00
max,2.000000e+03,9.900000e+01,2.200000e+01,1.000000e+00,1.000000e+00,1.000000e+00,0.0,4.100000e+01,4.500000e+02,2.000000e+03,1.300000e+01,2.380000e+02,1.770297e+05,1.788000e+05,1.527562e+05


In [ ]:
print("changing dates to time stamps")
training["membership_expire_date"] = training.membership_expire_date.apply(lambda x: datetime.strptime(str(x), "%Y-%m-%d").date() if pd.notnull(x) else x)
training["membership_expire_date"] = training.membership_expire_date.apply(lambda x: time.mktime(x.timetuple()) if pd.notnull(x) else 0.0)

training["transaction_date"] = training.transaction_date.apply(lambda x: datetime.strptime(str(x), "%Y-%m-%d").date() if pd.notnull(x) else x)
training["transaction_date"] = training.transaction_date.apply(lambda x: time.mktime(x.timetuple()) if pd.notnull(x) else 0.0)

training["registration_init_time"] = training.registration_init_time.apply(lambda x: datetime.strptime(str(x), "%Y-%m-%d").date() if pd.notnull(x) else x)
training["registration_init_time"] = training.registration_init_time.apply(lambda x: time.mktime(x.timetuple()) if pd.notnull(x) else 0.0)

testing["membership_expire_date"] = testing.membership_expire_date.apply(lambda x: datetime.strptime(str(x), "%Y-%m-%d").date() if pd.notnull(x) else x)
testing["membership_expire_date"] = testing.membership_expire_date.apply(lambda x: time.mktime(x.timetuple()) if pd.notnull(x) else 0.0)

testing["transaction_date"] = testing.transaction_date.apply(lambda x: datetime.strptime(str(x), "%Y-%m-%d").date() if pd.notnull(x) else x)
testing["transaction_date"] = testing.transaction_date.apply(lambda x: time.mktime(x.timetuple()) if pd.notnull(x) else 0.0)

testing["registration_init_time"] = testing.registration_init_time.apply(lambda x: datetime.strptime(str(x), "%Y-%m-%d").date() if pd.notnull(x) else x)
testing["registration_init_time"] = testing.registration_init_time.apply(lambda x: time.mktime(x.timetuple()) if pd.notnull(x) else 0.0)

In [ ]:
testing.describe()

In [ ]:
print("preprocessing")
X_train, y_train = preproc(training, mode='train', oneHot=False)
X_test, y_test = preproc(testing, mode="test", oneHot=False)

# parameters
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'binary_logloss'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

i = 0
K = 5
kf = KFold(n_splits=K, random_state=42, shuffle=True)

# training with KFold Cross Validation
results = []
weights = np.zeros(len(y_train))
weights[y_train == 0] = 1
weights[y_train == 1] = 30

In [ ]:
print('Start training...')
for train_index, test_index in kf.split(X_train):
    lgb_train = lgb.Dataset(X_train[train_index], y_train[train_index], weight=weights[train_index])
    lgb_eval = lgb.Dataset(X_train[test_index], y_train[test_index], reference=lgb_train)
    gbm = lgb.train(params,
        train_set=lgb_train,
        num_boost_round=200,
        valid_sets=lgb_eval,
        early_stopping_rounds=10,
        verbose_eval=5,
        feval=log_loss_lgbm)
    res = gbm.predict(X_test)
    i += 1
    results.append(res)

In [ ]:
submission = pd.DataFrame((results[0] + results[1] + results[2] + results[3] + results[4]) / 5)

In [ ]:
submission.columns = ["is_churn"]

In [ ]:
submission.describe()

In [ ]:
submission["msno"] = testing["msno"]

In [ ]:
submission.head()

In [ ]:
submission.to_csv('5Kfold_lgbm.csv', header=True, index=False)
print("created submission file")

In [ ]:
submission.describe()